In [1]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset, random_split
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.metrics import f1_score, classification_report
# import pandas as pd
# import numpy as np

# # Load data
# amplitude_df = pd.read_csv("/kaggle/input/amplifreq/all_signals.csv", header=None)
# frequency_df = pd.read_csv("/kaggle/input/amplifreq/all_frequencies.csv", header=None)

# # Step 1: Prepare amplitude input (X)
# X = amplitude_df.iloc[:, 1:].values.astype(np.float32)  # Keep time column in sync but skip it for training
# X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-6)  # Normalize each sample

# # Step 2: Prepare frequency labels (y)
# y_raw = frequency_df.iloc[:, 1:].values  # Skip first column only for labeling

# # Extract actual frequency values while keeping time 0s intact
# label_sets = []
# for row in y_raw:
#     freqs = [val for val in row if val not in [0, 50, 150, 250, 350, 450, 550, 650, 750, 881, 950]]  # Keep actual frequency values only
#     label_sets.append(list(set(freqs)))  # Unique labels per row

# # Multi-label binarization
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(label_sets)

# # Step 3: Tensor conversion
# X_tensor = torch.tensor(X)
# y_tensor = torch.tensor(y).float()

# # Step 4: Dataset and Dataloader
# dataset = TensorDataset(X_tensor, y_tensor)
# train_size = int(0.8 * len(dataset))
# train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64)

# # Step 5: Define model
# class FrequencyClassifier(nn.Module):
#     def __init__(self, input_size, num_labels):
#         super(FrequencyClassifier, self).__init__()
#         self.net = nn.Sequential(
#             nn.Linear(input_size, 512),
#             nn.ReLU(),
#             nn.BatchNorm1d(512),
#             nn.Dropout(0.3),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.BatchNorm1d(256),
#             nn.Dropout(0.3),
#             nn.Linear(256, num_labels),
#             nn.Sigmoid()  # Multi-label classification
#         )

#     def forward(self, x):
#         return self.net(x)

# # Initialize model
# model = FrequencyClassifier(X.shape[1], y.shape[1])
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Step 6: Training loop
# for epoch in range(350):
#     model.train()
#     total_loss = 0
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f"Epoch {epoch+1}/20, Loss: {total_loss / len(train_loader):.4f}")

# # Step 7: Evaluation
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for inputs, labels in val_loader:
#         outputs = model(inputs)
#         preds = (outputs > 0.5).int()
#         all_preds.extend(preds.numpy())
#         all_labels.extend(labels.numpy())

# f1 = f1_score(all_labels, all_preds, average='micro')
# print(f"\nMicro F1 Score: {f1:.4f}")
# print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=[str(label) for label in mlb.classes_]))


In [2]:
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
import pandas as pd
import numpy as np

# Load data
amplitude_df = pd.read_csv("/kaggle/input/secound/all_signals_1.csv", header=None)
frequency_df = pd.read_csv("/kaggle/input/secound/all_frequencies_1.csv", header=None)

# Step 1: Prepare amplitude input (X)
X = amplitude_df.iloc[:, 1:].values.astype(np.float32)
X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-6)

# Step 2: Prepare frequency labels (y)
y_raw = frequency_df.iloc[:, 1:].values
label_sets = [list(set(row[row != 0])) for row in y_raw]

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(label_sets)

# Step 3: Tensor conversion
X_tensor = torch.tensor(X).unsqueeze(1)  # Add channel dimension: (N, 1, T)
y_tensor = torch.tensor(y).float()

# Step 4: Dataset and Dataloader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Step 5: Define CNN model
class CNNClassifier(nn.Module):
    def __init__(self, input_length, num_labels):
        super(CNNClassifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, padding=2),  # (B, 32, T)
            nn.ReLU(),
            nn.MaxPool1d(2),  # (B, 32, T/2)
            nn.Conv1d(32, 64, kernel_size=3, padding=1),  # (B, 64, T/2)
            nn.ReLU(),
            nn.MaxPool1d(2),  # (B, 64, T/4)
        )
        self.flatten_dim = (input_length // 4) * 64
        self.fc = nn.Sequential(
            nn.Linear(self.flatten_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_labels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Instantiate CNN
model = CNNClassifier(X.shape[1], y.shape[1])

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 6: Training
for epoch in range(50):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/50, Loss: {total_loss / len(train_loader):.4f}")

# Step 7: Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        preds = (outputs > 0.5).int()
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

f1 = f1_score(all_labels, all_preds, average='micro')
print(f"Micro F1 Score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1/50, Loss: 0.2264
Epoch 2/50, Loss: 0.0347
Epoch 3/50, Loss: 0.0264
Epoch 4/50, Loss: 0.0232
Epoch 5/50, Loss: 0.0211
Epoch 6/50, Loss: 0.0200
Epoch 7/50, Loss: 0.0190
Epoch 8/50, Loss: 0.0177
Epoch 9/50, Loss: 0.0170
Epoch 10/50, Loss: 0.0160
Epoch 11/50, Loss: 0.0151
Epoch 12/50, Loss: 0.0146
Epoch 13/50, Loss: 0.0138
Epoch 14/50, Loss: 0.0136
Epoch 15/50, Loss: 0.0128
Epoch 16/50, Loss: 0.0122
Epoch 17/50, Loss: 0.0117
Epoch 18/50, Loss: 0.0111
Epoch 19/50, Loss: 0.0110
Epoch 20/50, Loss: 0.0102
Epoch 21/50, Loss: 0.0098
Epoch 22/50, Loss: 0.0096
Epoch 23/50, Loss: 0.0096
Epoch 24/50, Loss: 0.0089
Epoch 25/50, Loss: 0.0086
Epoch 26/50, Loss: 0.0082
Epoch 27/50, Loss: 0.0085
Epoch 28/50, Loss: 0.0080
Epoch 29/50, Loss: 0.0078
Epoch 30/50, Loss: 0.0076
Epoch 31/50, Loss: 0.0071
Epoch 32/50, Loss: 0.0071
Epoch 33/50, Loss: 0.0068
Epoch 34/50, Loss: 0.0067
Epoch 35/50, Loss: 0.0066
Epoch 36/50, Loss: 0.0066
Epoch 37/50, Loss: 0.0066
Epoch 38/50, Loss: 0.0061
Epoch 39/50, Loss: 0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [4]:
import pandas as pd

model.eval()
all_preds = []
all_labels = []
comparison_rows = []

threshold = 0.5  # You can tune this

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        probs = outputs  # Already sigmoid from model
        preds = (probs > threshold).int()
        
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())
        
        # Record comparison
        for p, t in zip(preds, labels):
            pred_labels = [mlb.classes_[i] for i, val in enumerate(p) if val == 1]
            true_labels = [mlb.classes_[i] for i, val in enumerate(t) if val == 1]
            comparison_rows.append({
                "Predicted": pred_labels,
                "True": true_labels
            })

# Show F1 score
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_preds, average='micro')
print(f"Micro F1 Score: {f1:.4f}")

# Convert comparisons to a DataFrame for easier viewing
comparison_df = pd.DataFrame(comparison_rows)
print(comparison_df.head(10))  # Show first 10 predictions and ground truths


comparison_df.to_csv("prediction_vs_ground_truth.csv", index=False)


Micro F1 Score: 0.9286
                                  Predicted  \
0                           [150, 250, 450]   
1  [150, 250, 350, 450, 550, 650, 750, 950]   
2                                        []   
3            [150, 250, 350, 450, 550, 650]   
4                      [150, 250, 350, 450]   
5            [150, 250, 350, 450, 550, 650]   
6                 [150, 250, 350, 450, 550]   
7  [150, 250, 363, 450, 550, 650, 750, 850]   
8                      [150, 250, 350, 550]   
9            [150, 250, 350, 450, 550, 750]   

                                            True  
0                      [150, 250, 377, 450, 550]  
1  [150, 250, 350, 450, 550, 650, 750, 878, 950]  
2                                             []  
3                 [150, 250, 350, 450, 550, 650]  
4                           [150, 250, 350, 450]  
5                 [150, 250, 350, 450, 550, 650]  
6                      [150, 250, 350, 450, 550]  
7       [150, 250, 364, 450, 550, 650, 750, 850]  


In [5]:
import shutil

# Move CSV to /kaggle/working if it's not already there
shutil.move("prediction_vs_ground_truth.csv", "/kaggle/working/prediction_vs_ground_truth.csv")


'/kaggle/working/prediction_vs_ground_truth.csv'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
import time

# ----------------------
# Training (CNN) — Track Time
# ----------------------
start_time = time.time()

for epoch in range(50):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/50, Loss: {total_loss / len(train_loader):.4f}")

end_time = time.time()
training_time_CNN = end_time - start_time
print(f"\n⏱️ Training Time (CNN): {training_time_CNN:.2f} seconds")


model.eval()
all_preds, all_labels = [], []

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = (outputs > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 5% error tolerance: allow some label mismatches
        total_labels = labels.size(1)
        mismatches = (preds != labels).sum(dim=1)  # per-sample mismatches
        allowed_errors = int(0.01*total_labels)

        correct += (mismatches <= allowed_errors).sum().item()
        total += labels.size(0)

# ----------------------
# Metrics
# ----------------------
f1_CNN = f1_score(all_labels, all_preds, average='micro')
accuracy_CNN = correct / total

print(f"\n✅ Micro F1 Score (CNN): {f1_CNN:.4f}")
print(f"🎯 Match Accuracy (≤1% Label Error): {accuracy_CNN:.4f}")
print(f"⏱️ Training Time (CNN): {training_time_CNN:.2f} seconds")

print("\n📋 Classification Report (CNN):\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1/50, Loss: 0.0046
Epoch 2/50, Loss: 0.0044
Epoch 3/50, Loss: 0.0047
Epoch 4/50, Loss: 0.0047
Epoch 5/50, Loss: 0.0045
Epoch 6/50, Loss: 0.0041
Epoch 7/50, Loss: 0.0041
Epoch 8/50, Loss: 0.0042
Epoch 9/50, Loss: 0.0043
Epoch 10/50, Loss: 0.0044
Epoch 11/50, Loss: 0.0039
Epoch 12/50, Loss: 0.0039
Epoch 13/50, Loss: 0.0037
Epoch 14/50, Loss: 0.0038
Epoch 15/50, Loss: 0.0038
Epoch 16/50, Loss: 0.0037
Epoch 17/50, Loss: 0.0038
Epoch 18/50, Loss: 0.0037
Epoch 19/50, Loss: 0.0038
Epoch 20/50, Loss: 0.0035
Epoch 21/50, Loss: 0.0036
Epoch 22/50, Loss: 0.0034
Epoch 23/50, Loss: 0.0035
Epoch 24/50, Loss: 0.0034
Epoch 25/50, Loss: 0.0033
Epoch 26/50, Loss: 0.0033
Epoch 27/50, Loss: 0.0032
Epoch 28/50, Loss: 0.0031
Epoch 29/50, Loss: 0.0029
Epoch 30/50, Loss: 0.0030
Epoch 31/50, Loss: 0.0031
Epoch 32/50, Loss: 0.0031
Epoch 33/50, Loss: 0.0033
Epoch 34/50, Loss: 0.0032
Epoch 35/50, Loss: 0.0030
Epoch 36/50, Loss: 0.0031
Epoch 37/50, Loss: 0.0030
Epoch 38/50, Loss: 0.0030
Epoch 39/50, Loss: 0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
import pandas as pd
import numpy as np

# Load data
amplitude_df = pd.read_csv("/kaggle/input/secound/all_signals_1.csv", header=None)
frequency_df = pd.read_csv("/kaggle/input/secound/all_frequencies_1.csv", header=None)

# Step 1: Prepare amplitude input (X)
X = amplitude_df.iloc[:, 1:].values.astype(np.float32)  # Skip timestamp
X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-6)

# Step 2: Prepare frequency labels (y)
y_raw = frequency_df.iloc[:, 1:].values
label_sets = [list(set(row[row != 0])) for row in y_raw]

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(label_sets)

# Step 3: Tensor conversion
X_tensor = torch.tensor(X).unsqueeze(1)  # Add channel dimension for SelfONN
y_tensor = torch.tensor(y).float()

# Step 4: Dataset and Dataloader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Step 5: Define SelfONN Layer
class SelfONNLayer1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, order=3, stride=1, padding=0):
        super(SelfONNLayer1D, self).__init__()
        self.order = order
        self.weights = nn.ParameterList([
            nn.Parameter(torch.randn(out_channels, in_channels, kernel_size))
            for _ in range(order)
        ])
        self.bias = nn.Parameter(torch.zeros(out_channels))
        self.stride = stride
        self.padding = padding

    def forward(self, x):
        out = 0
        for k in range(self.order):
            out += nn.functional.conv1d(x ** (k + 1), self.weights[k], bias=None,
                                        stride=self.stride, padding=self.padding)
        out += self.bias.view(1, -1, 1)
        return out

# Step 6: Define SelfONN Classifier
class SelfONNClassifier(nn.Module):
    def __init__(self, input_length, num_labels, order=3):
        super(SelfONNClassifier, self).__init__()
        self.conv1 = SelfONNLayer1D(1, 32, kernel_size=5, order=order, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = SelfONNLayer1D(32, 64, kernel_size=3, order=order, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(2)

        flat_dim = (input_length // 4) * 64

        self.fc1 = nn.Linear(flat_dim, 256)
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(256, num_labels)

    def forward(self, x):
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = nn.functional.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.drop(x)
        return torch.sigmoid(self.out(x))

# Instantiate model
input_length = X.shape[1]
num_labels = y.shape[1]
model = SelfONNClassifier(input_length=input_length, num_labels=num_labels, order=3)

# Step 7: Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 8: Training loop
for epoch in range(50):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/50, Loss: {total_loss / len(train_loader):.4f}")

# Step 9: Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        preds = (outputs > 0.5).int()
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

f1 = f1_score(all_labels, all_preds, average='micro')
print(f"Micro F1 Score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1/50, Loss: 0.1581
Epoch 2/50, Loss: 0.0419
Epoch 3/50, Loss: 0.0273
Epoch 4/50, Loss: 0.0253
Epoch 5/50, Loss: 0.0242
Epoch 6/50, Loss: 0.0229
Epoch 7/50, Loss: 0.0224
Epoch 8/50, Loss: 0.0217
Epoch 9/50, Loss: 0.0213
Epoch 10/50, Loss: 0.0204
Epoch 11/50, Loss: 0.0194
Epoch 12/50, Loss: 0.0187
Epoch 13/50, Loss: 0.0179
Epoch 14/50, Loss: 0.0173
Epoch 15/50, Loss: 0.0164
Epoch 16/50, Loss: 0.0153
Epoch 17/50, Loss: 0.0146
Epoch 18/50, Loss: 0.0135
Epoch 19/50, Loss: 0.0129
Epoch 20/50, Loss: 0.0118
Epoch 21/50, Loss: 0.0112
Epoch 22/50, Loss: 0.0106
Epoch 23/50, Loss: 0.0099
Epoch 24/50, Loss: 0.0096
Epoch 25/50, Loss: 0.0089
Epoch 26/50, Loss: 0.0088
Epoch 27/50, Loss: 0.0080
Epoch 28/50, Loss: 0.0078
Epoch 29/50, Loss: 0.0074
Epoch 30/50, Loss: 0.0071
Epoch 31/50, Loss: 0.0065
Epoch 32/50, Loss: 0.0066
Epoch 33/50, Loss: 0.0064
Epoch 34/50, Loss: 0.0060
Epoch 35/50, Loss: 0.0058
Epoch 36/50, Loss: 0.0058
Epoch 37/50, Loss: 0.0053
Epoch 38/50, Loss: 0.0052
Epoch 39/50, Loss: 0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [9]:
import time

# ----------------------
# Training (SONN) — Track Time
# ----------------------
start_time = time.time()

for epoch in range(50):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/50, Loss: {total_loss / len(train_loader):.4f}")

end_time = time.time()
training_time_SONN = end_time - start_time
print(f"\n⏱️ Training Time (SONN): {training_time_SONN:.2f} seconds")

# ----------------------
# Evaluation (5% Label Error Tolerance)
# ----------------------
model.eval()
all_preds, all_labels = [], []

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = (outputs > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 5% error tolerance: allow some label mismatches
        total_labels = labels.size(1)
        mismatches = (preds != labels).sum(dim=1)
        allowed_errors = int(0.01 * total_labels)

        correct += (mismatches <= allowed_errors).sum().item()
        total += labels.size(0)

# ----------------------
# Metrics
# ----------------------
f1_SONN = f1_score(all_labels, all_preds, average='micro')
accuracy_SONN = correct / total

print(f"\n✅ Micro F1 Score (SONN): {f1_SONN:.4f}")
print(f"🎯 Match Accuracy (≤1% Label Error): {accuracy_SONN:.4f}")
print(f"⏱️ Training Time (SONN): {training_time_SONN:.2f} seconds")

print("\n📋 Classification Report (SONN):\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1/50, Loss: 0.0034
Epoch 2/50, Loss: 0.0035
Epoch 3/50, Loss: 0.0034
Epoch 4/50, Loss: 0.0031
Epoch 5/50, Loss: 0.0031
Epoch 6/50, Loss: 0.0032
Epoch 7/50, Loss: 0.0030
Epoch 8/50, Loss: 0.0029
Epoch 9/50, Loss: 0.0029
Epoch 10/50, Loss: 0.0028
Epoch 11/50, Loss: 0.0027
Epoch 12/50, Loss: 0.0026
Epoch 13/50, Loss: 0.0028
Epoch 14/50, Loss: 0.0025
Epoch 15/50, Loss: 0.0026
Epoch 16/50, Loss: 0.0027
Epoch 17/50, Loss: 0.0025
Epoch 18/50, Loss: 0.0024
Epoch 19/50, Loss: 0.0024
Epoch 20/50, Loss: 0.0025
Epoch 21/50, Loss: 0.0022
Epoch 22/50, Loss: 0.0023
Epoch 23/50, Loss: 0.0024
Epoch 24/50, Loss: 0.0022
Epoch 25/50, Loss: 0.0021
Epoch 26/50, Loss: 0.0021
Epoch 27/50, Loss: 0.0021
Epoch 28/50, Loss: 0.0020
Epoch 29/50, Loss: 0.0020
Epoch 30/50, Loss: 0.0020
Epoch 31/50, Loss: 0.0020
Epoch 32/50, Loss: 0.0020
Epoch 33/50, Loss: 0.0020
Epoch 34/50, Loss: 0.0019
Epoch 35/50, Loss: 0.0019
Epoch 36/50, Loss: 0.0018
Epoch 37/50, Loss: 0.0019
Epoch 38/50, Loss: 0.0018
Epoch 39/50, Loss: 0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [10]:
# pip install snntorch


In [11]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset, random_split
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.metrics import f1_score, classification_report
# import pandas as pd
# import numpy as np
# import snntorch as snn
# from snntorch import surrogate
# from snntorch import functional as SF

# # Load and normalize data
# amplitude_df = pd.read_csv("/kaggle/input/amplifreq/all_signals.csv", header=None)
# frequency_df = pd.read_csv("/kaggle/input/amplifreq/all_frequencies.csv", header=None)

# X = amplitude_df.iloc[:, 1:].values.astype(np.float32)
# X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-6)
# y_raw = frequency_df.iloc[:, 1:].values
# label_sets = [list(set(row[row != 0])) for row in y_raw]
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(label_sets)

# X_tensor = torch.tensor(X).unsqueeze(1)  # (B, C=1, T)
# y_tensor = torch.tensor(y).float()

# dataset = TensorDataset(X_tensor, y_tensor)
# train_size = int(0.8 * len(dataset))
# train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64)

# # SNN Classifier
# class SNNClassifier(nn.Module):
#     def __init__(self, input_length, num_labels, time_window=20):
#         super().__init__()
#         self.time_window = time_window
#         self.conv1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
#         self.lif1 = snn.Leaky(beta=0.95, spike_grad=surrogate.fast_sigmoid())

#         self.pool1 = nn.MaxPool1d(2)

#         self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
#         self.lif2 = snn.Leaky(beta=0.95, spike_grad=surrogate.fast_sigmoid())
#         self.pool2 = nn.MaxPool1d(2)

#         flat_dim = (input_length // 4) * 64
#         self.fc1 = nn.Linear(flat_dim, 256)
#         self.lif3 = snn.Leaky(beta=0.95, spike_grad=surrogate.fast_sigmoid())

#         self.fc2 = nn.Linear(256, num_labels)

#     def forward(self, x):
#         spk_rec = []
#         mem1 = mem2 = mem3 = 0

#         for step in range(self.time_window):
#             cur_input = x + torch.rand_like(x) * 0.01  # small noise simulating input variation

#             x1 = self.conv1(cur_input)
#             spk1, mem1 = self.lif1(x1, mem1)
#             x1 = self.pool1(spk1)

#             x2 = self.conv2(x1)
#             spk2, mem2 = self.lif2(x2, mem2)
#             x2 = self.pool2(spk2)

#             x2 = x2.view(x2.size(0), -1)
#             x3 = self.fc1(x2)
#             spk3, mem3 = self.lif3(x3, mem3)

#             out = self.fc2(spk3)
#             spk_rec.append(out)

#         spk_rec = torch.stack(spk_rec, dim=0)  # (time, batch, labels)
#         out_rate = spk_rec.mean(dim=0)  # firing rate
#         return torch.sigmoid(out_rate)

# # Instantiate model
# input_length = X.shape[1]
# num_labels = y.shape[1]
# model = SNNClassifier(input_length=input_length, num_labels=num_labels, time_window=20)

# # Training Setup
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training Loop
# for epoch in range(20):  # Reduce epochs due to longer simulation
#     model.train()
#     total_loss = 0
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# # Evaluation
# model.eval()
# all_preds = []
# all_labels = []
# with torch.no_grad():
#     for inputs, labels in val_loader:
#         outputs = model(inputs)
#         preds = (outputs > 0.5).int()
#         all_preds.extend(preds.numpy())
#         all_labels.extend(labels.numpy())

# f1 = f1_score(all_labels, all_preds, average='micro')
# print(f"Micro F1 Score: {f1:.4f}")
# print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
import pandas as pd
import numpy as np

# ----------------------
# Load and preprocess data
# ----------------------
amplitude_df = pd.read_csv("/kaggle/input/secound/all_signals_1.csv", header=None)
frequency_df = pd.read_csv("/kaggle/input/secound/all_frequencies_1.csv", header=None)

X = amplitude_df.iloc[:, 1:].values.astype(np.float32)
X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-6)

y_raw = frequency_df.iloc[:, 1:].values
label_sets = [list(set(row[row != 0])) for row in y_raw]
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(label_sets)

X_tensor = torch.tensor(X).unsqueeze(1)  # shape: [N, 1, T]
y_tensor = torch.tensor(y).float()

# Optionally reduce dataset size for memory reasons
X_tensor = X_tensor[:2000]
y_tensor = y_tensor[:2000]

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# ----------------------
# Define Spiking Neuron
# ----------------------
class LIFNeuron(nn.Module):
    def __init__(self, threshold=1.0, decay=0.9):
        super().__init__()
        self.threshold = threshold
        self.decay = decay

    def forward(self, x):
        # x shape: [batch, channels, time]
        batch_size, channels, time = x.size()
        mem = torch.zeros(batch_size, channels, device=x.device)
        spikes = []

        for t in range(time):
            input_t = x[:, :, t]  # shape: [batch, channels]
            mem = self.decay * mem + input_t
            spike = (mem >= self.threshold).float()
            mem = mem * (1 - spike)  # reset after spike
            spikes.append(spike.unsqueeze(-1))  # shape: [batch, channels, 1]

        return torch.cat(spikes, dim=-1)  # shape: [batch, channels, time]


# ----------------------
# SNN Model Definition
# ----------------------
class SNNClassifier(nn.Module):
    def __init__(self, input_length, num_labels):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
        self.lif1 = LIFNeuron()
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.lif2 = LIFNeuron()
        self.pool2 = nn.MaxPool1d(2)

        flat_dim = (input_length // 4) * 64
        self.fc1 = nn.Linear(flat_dim, 256)
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(256, num_labels)

    def forward(self, x):
        x = self.pool1(self.lif1(self.conv1(x)))
        x = self.pool2(self.lif2(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.drop(x)
        return torch.sigmoid(self.out(x))

# ----------------------
# Train & Evaluate
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_length = X_tensor.shape[2]
num_labels = y_tensor.shape[1]
model = SNNClassifier(input_length, num_labels).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):  # Fewer epochs for memory safety
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# ----------------------
# Evaluation
# ----------------------
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = (outputs.cpu() > 0.5).int()
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

f1 = f1_score(all_labels, all_preds, average='micro')
print(f"\nMicro F1 Score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1, Loss: 0.0883
Epoch 2, Loss: 0.0233
Epoch 3, Loss: 0.0183
Epoch 4, Loss: 0.0157
Epoch 5, Loss: 0.0135
Epoch 6, Loss: 0.0114
Epoch 7, Loss: 0.0097
Epoch 8, Loss: 0.0082
Epoch 9, Loss: 0.0070
Epoch 10, Loss: 0.0059
Epoch 11, Loss: 0.0050
Epoch 12, Loss: 0.0043
Epoch 13, Loss: 0.0038
Epoch 14, Loss: 0.0033
Epoch 15, Loss: 0.0027
Epoch 16, Loss: 0.0026
Epoch 17, Loss: 0.0022
Epoch 18, Loss: 0.0020
Epoch 19, Loss: 0.0019
Epoch 20, Loss: 0.0016
Epoch 21, Loss: 0.0016
Epoch 22, Loss: 0.0014
Epoch 23, Loss: 0.0013
Epoch 24, Loss: 0.0012
Epoch 25, Loss: 0.0011
Epoch 26, Loss: 0.0011
Epoch 27, Loss: 0.0010
Epoch 28, Loss: 0.0009
Epoch 29, Loss: 0.0009
Epoch 30, Loss: 0.0008
Epoch 31, Loss: 0.0008
Epoch 32, Loss: 0.0008
Epoch 33, Loss: 0.0007
Epoch 34, Loss: 0.0007
Epoch 35, Loss: 0.0007
Epoch 36, Loss: 0.0006
Epoch 37, Loss: 0.0006
Epoch 38, Loss: 0.0006
Epoch 39, Loss: 0.0006
Epoch 40, Loss: 0.0005
Epoch 41, Loss: 0.0006
Epoch 42, Loss: 0.0005
Epoch 43, Loss: 0.0004
Epoch 44, Loss: 0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [13]:
amplitude_df.shape

(2000, 801)

In [14]:
import time

# ----------------------
# Training with time tracking
# ----------------------
start_time = time.time()

for epoch in range(50):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/50, Loss: {total_loss / len(train_loader):.4f}")

end_time = time.time()
training_time_SENN = end_time - start_time
print(f"\n⏱️ Training Time: {training_time_SENN:.2f} seconds")

# ----------------------
# Evaluation (5% Label Error Tolerance)
# ----------------------
model.eval()
all_preds, all_labels = [], []

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = (outputs > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 5% error tolerance: allow some label mismatches
        total_labels = labels.size(1)
        mismatches = (preds != labels).sum(dim=1)  # per-sample mismatches
        allowed_errors = int(0.01 * total_labels)

        correct += (mismatches <= allowed_errors).sum().item()
        total += labels.size(0)

# Micro F1 Score
f1_SENN = f1_score(all_labels, all_preds, average='micro')
accuracy_SENN = correct / total

print(f"\n✅ Micro F1 Score: {f1_SENN:.4f}")
print(f"🎯 Match Accuracy with ≤1% Label Error: {accuracy_SENN:.4f}")
print(f"⏱️ Training Time: {training_time_SENN:.2f} seconds")

print("\n📋 Classification Report:\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1/50, Loss: 0.0004
Epoch 2/50, Loss: 0.0004
Epoch 3/50, Loss: 0.0004
Epoch 4/50, Loss: 0.0004
Epoch 5/50, Loss: 0.0003
Epoch 6/50, Loss: 0.0003
Epoch 7/50, Loss: 0.0003
Epoch 8/50, Loss: 0.0004
Epoch 9/50, Loss: 0.0003
Epoch 10/50, Loss: 0.0003
Epoch 11/50, Loss: 0.0003
Epoch 12/50, Loss: 0.0003
Epoch 13/50, Loss: 0.0003
Epoch 14/50, Loss: 0.0003
Epoch 15/50, Loss: 0.0003
Epoch 16/50, Loss: 0.0003
Epoch 17/50, Loss: 0.0003
Epoch 18/50, Loss: 0.0003
Epoch 19/50, Loss: 0.0003
Epoch 20/50, Loss: 0.0003
Epoch 21/50, Loss: 0.0003
Epoch 22/50, Loss: 0.0003
Epoch 23/50, Loss: 0.0003
Epoch 24/50, Loss: 0.0003
Epoch 25/50, Loss: 0.0003
Epoch 26/50, Loss: 0.0003
Epoch 27/50, Loss: 0.0003
Epoch 28/50, Loss: 0.0003
Epoch 29/50, Loss: 0.0003
Epoch 30/50, Loss: 0.0002
Epoch 31/50, Loss: 0.0002
Epoch 32/50, Loss: 0.0003
Epoch 33/50, Loss: 0.0003
Epoch 34/50, Loss: 0.0003
Epoch 35/50, Loss: 0.0002
Epoch 36/50, Loss: 0.0003
Epoch 37/50, Loss: 0.0002
Epoch 38/50, Loss: 0.0002
Epoch 39/50, Loss: 0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
import pandas as pd
import numpy as np

# ------------------------------
# 1. Load and Normalize Data
# ------------------------------
amplitude_df = pd.read_csv("/kaggle/input/secound/all_signals_1.csv", header=None)
frequency_df = pd.read_csv("/kaggle/input/secound/all_frequencies_1.csv", header=None)

X = amplitude_df.iloc[:, 1:].values.astype(np.float32)  # Skip timestamp column
X = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-6)

# ------------------------------
# 2. Pad/Truncate and Reshape to (N, 1, 128, 128)
# ------------------------------
target_len = 128 * 128
X_fixed = []

for row in X:
    if len(row) < target_len:
        padded = np.pad(row, (0, target_len - len(row)), mode='constant')
    else:
        padded = row[:target_len]
    X_fixed.append(padded)

X_fixed = np.array(X_fixed).reshape(-1, 1, 128, 128).astype(np.float32)

# ------------------------------
# 3. Prepare Multi-Label Targets
# ------------------------------
y_raw = frequency_df.iloc[:, 1:].values  # Skip timestamp column
label_sets = [list(set(row[row != 0])) for row in y_raw]

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(label_sets)

# ------------------------------
# 4. Create Datasets and Loaders
# ------------------------------
X_tensor = torch.tensor(X_fixed)
y_tensor = torch.tensor(y).float()

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# ------------------------------
# 5. Define CNN-based Multi-Label Classifier
# ------------------------------
class CNNFrequencyClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNFrequencyClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
            nn.Sigmoid()  # For multi-label classification
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = CNNFrequencyClassifier(num_classes=y.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------
# 6. Training Loop
# ------------------------------
for epoch in range(10):  # You can increase if needed
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/10, Loss: {total_loss / len(train_loader):.4f}")

# ------------------------------
# 7. Evaluation
# ------------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        preds = (outputs > 0.5).int()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

f1 = f1_score(all_labels, all_preds, average='micro')
print(f"\n✅ Micro F1 Score: {f1:.4f}")
print("\n📋 Classification Report:\n", classification_report(all_labels, all_preds, target_names=[str(l) for l in mlb.classes_]))


Epoch 1/10, Loss: 0.1193
Epoch 2/10, Loss: 0.0359
Epoch 3/10, Loss: 0.0320
Epoch 4/10, Loss: 0.0263
Epoch 5/10, Loss: 0.0239
Epoch 6/10, Loss: 0.0229
Epoch 7/10, Loss: 0.0217
Epoch 8/10, Loss: 0.0205
Epoch 9/10, Loss: 0.0199
Epoch 10/10, Loss: 0.0191

✅ Micro F1 Score: 0.8640

📋 Classification Report:
               precision    recall  f1-score   support

         150       0.96      0.98      0.97       318
         160       0.00      0.00      0.00         2
         161       0.00      0.00      0.00         0
         162       0.00      0.00      0.00         1
         163       0.00      0.00      0.00         0
         164       0.00      0.00      0.00         0
         165       0.00      0.00      0.00         3
         166       0.00      0.00      0.00         1
         167       0.00      0.00      0.00         2
         168       0.00      0.00      0.00         1
         169       0.00      0.00      0.00         3
         170       0.00      0.00      0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.